# Making a dataset from Quandl and Wikipedia

In this notebook I have acquiring a dataset to predict/classify housing price in USA. 

The data is primarily from [Quandl]( https://www.quandl.com) and has the following information: 

1. [Housing price Index ]( https://www.quandl.com/data/FMAC?keyword=housing%20price%20index)from Freddie Mac. 
2.


In [2]:
import quandl
import pandas as pd
f = open('auth.txt', 'r')
api_key = f.read()
from IPython.display import HTML

In [3]:
HTML('<iframe src = https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States width=700 height=500></iframe>')

#### as can be seein in the folowing image, the data set for each state has a uniqe code which is consist of "FMAC/HPI_" + avriviaiton for the state name. 

![title]("img/picture.png")

In [4]:
fiddy_states = pd.read_html('https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States')
fiddy_states

[                       0       1               2               3  \
 0   State & abbreviation  Cities    Statehood[A]      Population   
 1                    NaN     NaN         Capital  Largest[C][17]   
 2                Alabama      AL      Montgomery      Birmingham   
 3                 Alaska      AK          Juneau       Anchorage   
 4                Arizona      AZ         Phoenix   Feb. 14, 1912   
 5               Arkansas      AR     Little Rock   Jun. 15, 1836   
 6             California      CA      Sacramento     Los Angeles   
 7               Colorado      CO          Denver    Aug. 1, 1876   
 8            Connecticut      CT        Hartford      Bridgeport   
 9               Delaware      DE           Dover      Wilmington   
 10               Florida      FL     Tallahassee    Jacksonville   
 11               Georgia      GA         Atlanta    Jan. 2, 1788   
 12                Hawaii      HI        Honolulu   Aug. 21, 1959   
 13                 Idaho      ID 

In [5]:
state_list = fiddy_states[0][1][2:]
state_list.head()

2    AL
3    AK
4    AZ
5    AR
6    CA
Name: 1, dtype: object

In [6]:
fiddy_states = pd.read_html('https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States')
state_list = fiddy_states[0][1][2:]

df1 = pd.DataFrame()

for abbv in state_list:
    query = "FMAC/HPI_"+str(abbv)
    df = quandl.get(query, authtoken=api_key)
    df.rename(columns={'Value':str(abbv)}, inplace=True)
    df[abbv] = (df[abbv]-df[abbv][0]) / df[abbv][0] * 100.0
    if df1.empty:
        df1 = df
    else:
        df1 = df1.join(df)

In [7]:
df1.head()

,AL,AK,AZ,AR,CA,CO,CT,DE,FL,GA,...,SD,TN,TX,UT,VT,VA,WA,WV,WI,WY
Date,,,,,,,,,,,,,,,,,,,,,
1975-01-31,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1975-02-28,0.626500,1.485775,1.688504,0.846192,0.356177,1.159639,2.123926,0.142451,3.938796,-0.902841,...,-1.099859,0.112132,0.951488,1.398290,0.945247,0.987288,0.397561,2.495069,1.404920,1.438502
1975-03-31,1.358575,3.006473,3.261346,1.581956,1.575690,2.299449,3.719898,0.387918,9.798243,-1.282758,...,-2.218400,0.369186,2.701039,2.659262,1.857227,1.707474,1.015723,5.093474,2.791371,2.962512
1975-04-30,2.254726,4.593530,4.475810,2.183669,3.573196,3.359028,4.616778,0.891619,16.974819,-1.068371,...,-3.328868,0.765570,5.276570,3.662119,2.685578,2.238392,1.757887,7.772685,4.034165,4.545011
1975-05-31,3.107829,6.327600,5.139617,2.786248,5.241395,4.226895,4.901787,1.752086,17.891884,-0.676830,...,-4.335377,1.228808,6.099286,4.326186,3.413295,2.684036,2.426439,10.469784,5.047628,6.080901


In [8]:
df2 = quandl.get("FMAC/HPI_USA", authtoken=api_key)
df2["Value_added"] = (df2["Value"]-df2["Value"][0]) / df2["Value"][0] * 100.0
df2.rename(columns={'Value_added':'US_House_Price_Index'}, inplace=True)
df2.head()

,Value,US_House_Price_Index
Date,,
1975-01-31,23.537716,0.000000
1975-02-28,23.688245,0.639523
1975-03-31,23.933544,1.681678
1975-04-30,24.254920,3.047044
1975-05-31,24.460992,3.922540


In [11]:
df6 = quandl.get("WORLDBANK/USA_SP_RUR_TOTL", trim_start="1974-12-30", authtoken=api_key)
df6.rename(columns={'Value':'WorldBankPopulation'}, inplace=True)
df6.tail()

,WorldBankPopulation
Date,
2010-12-31,59481252.0
2011-12-31,59414143.0
2012-12-31,59342054.0
2013-12-31,59257833.0
2014-12-31,59157550.0


In [9]:
df3 = quandl.get("FMAC/MORTG", trim_start="1975-01-01", authtoken=api_key)
df3["Value"] = (df3["Value"]-df3["Value"][0]) / df3["Value"][0] * 100.0
df3.rename(columns={'Value':'US_Mortgage'}, inplace=True)
df3=df3.resample('1D')
df3=df3.resample('M')
df3.head()

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: FutureWarning: .resample() is now a deferred operation
use .resample(...).mean() instead of .resample(...)
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:6: FutureWarning: .resample() is now a deferred operation
use .resample(...).mean() instead of .resample(...)


,US_Mortgage
Date,
1975-01-31,0.000000
1975-02-28,-3.393425
1975-03-31,-5.620361
1975-04-30,-6.468717
1975-05-31,-5.514316


In [10]:
df4 = quandl.get("YAHOO/INDEX_GSPC", trim_start="1975-01-01", authtoken=api_key)
df4["Adjusted Close"] = (df4["Adjusted Close"]-df4["Adjusted Close"][0]) / df4["Adjusted Close"][0] * 100.0
df4.rename(columns={'Adjusted Close':'sp500'}, inplace=True)
df4 = df4['sp500']
df4=df4.resample('M')
df4.head()

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:6: FutureWarning: .resample() is now a deferred operation
use .resample(...).mean() instead of .resample(...)


Date
1975-01-31     3.323491
1975-02-28    14.049322
1975-03-31    19.367785
1975-04-30    20.636734
1975-05-31    28.287322
Freq: M, Name: sp500, dtype: float64